# Indeed Web Crawler

In [29]:
# Import Libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [30]:
# Define Data Jobs List
data_jobs = ["Machine Learning Engineer", "Data Scientist", "Data Analyst", "Data Engineer", "Database Administrator", "Data Architect", "Statistician", "Business Intelligence Developer", "Data and Analytics Manager", "Data and Analytics Director"]

In [31]:
# Define Countries
countries = {
    "UK": "https://www.indeed.co.uk/jobs?q=data&l=",
    "France": "https://www.indeed.fr/jobs?q=data&l=",
    "Germany": "https://www.indeed.de/jobs?q=data&l=",
    "Luxembourg": "https://www.indeed.lu/jobs?q=data&l=",
    "Netherlands": "https://www.indeed.nl/jobs?q=data&l="
}

countries

{'UK': 'https://www.indeed.co.uk/jobs?q=data&l=',
 'France': 'https://www.indeed.fr/jobs?q=data&l=',
 'Germany': 'https://www.indeed.de/jobs?q=data&l=',
 'Luxembourg': 'https://www.indeed.lu/jobs?q=data&l=',
 'Netherlands': 'https://www.indeed.nl/jobs?q=data&l='}

In [32]:
def fetch_job_data(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36'
    }

    response = requests.get(url, headers=headers)
    print(f"Status code: {response.status_code}")  # 输出状态码
    if response.status_code == 200:
        return response.text
    else:
        print(f"Failed to retrieve data from {url}")
        return None

In [33]:
# 定义解析网页数据的函数
def parse_job_data(html):
    soup = BeautifulSoup(html, 'html.parser')
    job_list = []
    
    # 找到所有职位信息
    jobs = soup.find_all('div', class_='job_seen_beacon')
    for job in jobs:
        title = job.find('h2', class_='jobTitle').text if job.find('h2', class_='jobTitle') else 'N/A'
        company = job.find('span', class_='companyName').text if job.find('span', class_='companyName') else 'N/A'
        location = job.find('div', class_='companyLocation').text if job.find('div', class_='companyLocation') else 'N/A'
        
        job_list.append({
            'title': title,
            'company': company,
            'location': location
        })
    
    return job_list

In [34]:
# 将获取的职位数据保存到CSV文件
def save_to_csv(job_data, filename='data_jobs.csv'):
    df = pd.DataFrame(job_data)
    df.to_csv(filename, index=False, encoding='utf-8')
    print(f"Job data saved to {filename}")

In [35]:
fetch_job_data("https://www.indeed.co.uk/jobs?q=data&l=")

Status code: 403
Failed to retrieve data from https://www.indeed.co.uk/jobs?q=data&l=


In [36]:
def main():
    all_jobs = []
    for country, url in countries.items():
        print(f"Fetching jobs from {country}...")
        html = fetch_job_data(url)
        if html:
            jobs = parse_job_data(html)
            all_jobs.extend(jobs)
        time.sleep(5)  # 每次请求之间暂停5秒

In [55]:
import os

profile_path = '/home/yutao/snap/firefox/common/.mozilla/firefox/fwm9ujwo.yutao'  # 更新为您的实际路径
print(os.path.exists(profile_path))  # 检查路径是否存在
print(os.path.isfile(os.path.join(profile_path, 'prefs.js')))  # 检查 prefs.js 文件是否存在

True
True


In [57]:
from selenium import webdriver
import time

# 使用已定义的 profile_path 和 firefox_options
firefox_options.set_preference("profile", profile_path)

# 取消注释以下行以启用无头模式
# firefox_options.add_argument('--headless')  # 以无头模式运行

# 创建 Firefox 驱动实例
driver = webdriver.Firefox(service=service, options=firefox_options)

try:
    # 访问页面
    driver.get(url)

    # 等待页面加载
    time.sleep(3)  # 根据需要调整等待时间

    # 获取页面内容
    html = driver.page_source
    print(html)  # 打印获取的 HTML 内容
finally:
    # 关闭浏览器
    driver.quit()


WebDriverException: Message: Process unexpectedly closed with status 1


In [64]:
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.firefox.firefox_profile import FirefoxProfile

# 指定 Firefox 配置文件路径
profile_path = '/home/yutao/snap/firefox/common/.mozilla/firefox/fwm9ujwo.yutao'  # 替换为您的配置文件路径

# 配置 geckodriver
service = Service('/usr/local/bin/geckodriver')  # 指定 geckodriver 的路径

# 创建 Firefox 配置文件
profile = FirefoxProfile(profile_path)

# 配置 Firefox 选项
options = Options()
options.profile = profile
# options.add_argument('--headless')  # 如果需要无头模式，请取消注释

# 创建 Firefox 驱动实例
driver = webdriver.Firefox(options=options, service=service)

# 访问页面
url = "https://www.indeed.co.uk/jobs?q=data&l=London"
driver.get(url)

# 等待页面加载
driver.implicitly_wait(10)  # 等待页面加载（可根据需要调整）

# 获取页面内容
html = driver.page_source
print(html)  # 打印获取的 HTML 内容

# 关闭浏览器
driver.quit()

WebDriverException: Message: Process unexpectedly closed with status 1


In [65]:
import os 
import selenium.webdriver as webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options

user_argent = "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:131.0) Gecko/20100101 Firefox/131.0"

firefox_driver = os.path.join(os.getcwd(), "Drivers", )


In [67]:
from selenium import webdriver
from selenium.webdriver.common.by import By

driver = webdriver.Chrome()

driver.get("https://www.indeed.co.uk/jobs?q=data&l=")

title = driver.title

driver.implicitly_wait(0.5)

text_box = driver.find_element(by=By.NAME, value="my-text")
submit_button = driver.find_element(by=By.CSS_SELECTOR, value="button")

text_box.send_keys("Selenium")
submit_button.click()

message = driver.find_element(by=By.ID, value="message")
text = message.text

driver.quit()

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=129.0.6668.70)
Stacktrace:
#0 0x5ca492f4db9a <unknown>
#1 0x5ca492c33670 <unknown>
#2 0x5ca492c091df <unknown>
#3 0x5ca492cb009d <unknown>
#4 0x5ca492cc6689 <unknown>
#5 0x5ca492ca7923 <unknown>
#6 0x5ca492c756e7 <unknown>
#7 0x5ca492c766de <unknown>
#8 0x5ca492f1766b <unknown>
#9 0x5ca492f1b611 <unknown>
#10 0x5ca492f034e5 <unknown>
#11 0x5ca492f1c192 <unknown>
#12 0x5ca492ee86ef <unknown>
#13 0x5ca492f3c9d8 <unknown>
#14 0x5ca492f3cba7 <unknown>
#15 0x5ca492f4c9ec <unknown>
#16 0x796f06c94ac3 <unknown>
